# Imports

In [1]:
from qiskit.circuit.library.standard_gates import RXGate, RZGate, RYGate, CXGate, CZGate, SGate, HGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from typing import Dict, Tuple, List
import numpy as np
from numpy import linalg as LA
from qiskit.opflow import X, Z, I, H
from qiskit.quantum_info import Pauli
import matplotlib
from qiskit import assemble, Aer
from qiskit import *
from qiskit.visualization import plot_histogram
import math
from qiskit.algorithms.optimizers import L_BFGS_B, COBYLA

# Implementing the ansatz from qiskit tutorial

In [2]:
def calc_linear_entangelment_ansatz(num_of_qubits, thetas, depth):
    if thetas.size != 2*num_of_qubits*(depth + 1):
        print("ERROR with thetas size")
    else:
        qr = QuantumRegister(num_of_qubits, name="qubit")
        qc = QuantumCircuit(qr)
        
        for iteration in range(depth):
            for qubit_index in range(num_of_qubits):
                RY_theta_index = iteration*2*num_of_qubits + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
            for qubit_index in range(num_of_qubits - 1):
                qc.append(CXGate(), [qr[qubit_index], qr[qubit_index + 1]])
        
        for qubit_index in range(num_of_qubits):
                RY_theta_index = 2*num_of_qubits*depth + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
        
        return qc

def calc_full_entangelment_ansatz(num_of_qubits, thetas, depth):
    if thetas.size != 2*num_of_qubits*(depth + 1):
        print("ERROR with thetas size")
    else:
        qr = QuantumRegister(num_of_qubits, name="qubit")
        qc = QuantumCircuit(qr)
        
        for iteration in range(depth):
            for qubit_index in range(num_of_qubits):
                RY_theta_index = iteration*2*num_of_qubits + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
            for qubit_index in range(num_of_qubits - 1):
                for target_qubit_index in range(qubit_index + 1, num_of_qubits):
                    qc.append(CXGate(), [qr[qubit_index], qr[target_qubit_index]])
        
        for qubit_index in range(num_of_qubits):
                RY_theta_index = 2*num_of_qubits*depth + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
        
        return qc

def calc_ansatz_circ(num_of_qubits, entangelment, thetas, depth):
    if entangelment == "linear":
        return calc_linear_entangelment_ansatz(num_of_qubits, thetas, depth)
    assert entangelment == "full", "entangelment has to be linear or full"
    return calc_full_entangelment_ansatz(num_of_qubits, thetas, depth)

# Generating the hamiltonians

In [3]:
H2_molecule_Hamiltonian = -0.5053051899926562*(I^I) + \
                            -0.3277380754984016*(Z^I) + \
                            0.15567463610622564*(Z^Z) + \
                            -0.3277380754984016*(I^Z)

def create_pauli_string_with_pauli_op_on_index_i(pauli_op, i, qubits_num):
    if i == 1:
        pauli_string = pauli_op
        for qubit in range(qubits_num - 1):
            pauli_string = pauli_string ^ I
        return pauli_string
    
    pauli_string = I
    for qubit in range(2, qubits_num + 1):
        if qubit == i:
            pauli_string = pauli_string ^ pauli_op
        else:
            pauli_string = pauli_string ^ I
            
    return pauli_string

def create_pauli_string_with_pauli_ops_on_index_i_and_j(pauli_op_second, i, pauli_op_first, j, qubits_num):
    if j == 1:
        pauli_string = pauli_op_first
        for qubit in range(2, qubits_num + 1):
            if qubit == i:
                pauli_string = pauli_string ^ pauli_op_second
            else:
                pauli_string = pauli_string ^ I
        return pauli_string
    
    pauli_string = I
    for qubit in range(2, qubits_num + 1):
        if qubit == j:
            pauli_string = pauli_string ^ pauli_op_first
        elif qubit == i:
            pauli_string = pauli_string ^ pauli_op_second
        else:
            pauli_string = pauli_string ^ I
            
    return pauli_string
    
def get_Ising_model_hamiltonian():
    hamiltonian = I
    for qubit in range(QUBITS_NUM - 1):
        hamiltonian = hamiltonian^I
    hamiltonian = 0 * hamiltonian
    
    for i in range(1, QUBITS_NUM + 1):
        a_i = np.random.random_sample()
        x_i = create_pauli_string_with_pauli_op_on_index_i(X, i, QUBITS_NUM)
        hamiltonian = hamiltonian + a_i * x_i
        for j in range(1, i):
            J_ij = np.random.random_sample()
            z_ij = create_pauli_string_with_pauli_ops_on_index_i_and_j(Z, i, Z, j, QUBITS_NUM)
            hamiltonian = hamiltonian + J_ij * z_ij
    
    return hamiltonian
    
def get_hamiltonian(hamiltonian_type):
    if hamiltonian_type == "Ising Model":
        return get_Ising_model_hamiltonian()
    assert hamiltonian_type == "Molecular", "hamiltonian_type has to be Ising Model or Molecular"
    return H2_molecule_Hamiltonian

# Expectation Value

In [4]:
reducing_to_pauli_z_dict = {
    Pauli('I'): Pauli('I'),
    Pauli('Z'): Pauli('Z'),
    Pauli('X'): Pauli('Z'),
    Pauli('Y'): Pauli('Z')
} 

def transfrom_hamiltonian_into_pauli_string(hamiltonian):
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    pauli_coeff = list(map(lambda pauli_operator: pauli_operator.coeff, pauli_operators))
    return (pauli_strings, pauli_coeff)

def reduce_pauli_matrixes_into_sigma_z(pauli_string):
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = pauli_string[matrix_index]
        pauli_string[matrix_index].insert(reducing_to_pauli_z_dict[pauli_matrix])
    
    print(pauli_string)
    return pauli_string

def get_z_reduction_for_pauli_string(qc, pauli_string):
    qr = QuantumRegister(QUBITS_NUM, name="q")
    exdend_qc = QuantumCircuit(qr)
    pauli_string = str(pauli_string)
    for qubit_index, pauli_matrix in enumerate(pauli_string):
        if pauli_matrix == "X":
            exdend_qc.append(HGate(), [qr[qubit_index]])
        elif pauli_matrix == "Y":
            exdend_qc.append(HGate(), [qr[qubit_index]])
            exdend_qc.append(SGate(), [qr[qubit_index]])
    qc = qc.compose(exdend_qc)
    return qc

def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit) -> Dict:
    quantum_state_circuit.measure_all()

    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, simulator_backend) 
    Qobj = assemble(transpiled_quantum_state_circuit)
    result = simulator_backend.run(Qobj).result() 
    counts = result.get_counts(quantum_state_circuit)
    
    return get_probability_distribution(counts)

def sort_probas_dict_by_qubits_string_keys(proba_distribution: Dict) -> Dict:
    return dict(sorted(proba_distribution.items()))

def reset_power_of_minus_1(power_of_minus_1):
    power_of_minus_1 = 0
    return power_of_minus_1

def calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, probas_distribution):
    pauli_string_expectation_value = 0
    power_of_minus_1 = 0
    
    sorted_probas_distribuition = sort_probas_dict_by_qubits_string_keys(probas_distribution)
    for qubits_string, proba in sorted_probas_distribuition.items():
        for string_index in range(QUBITS_NUM):
            if(str(qubits_string[string_index])=="1" and str(pauli_string[string_index])=="Z"):
                power_of_minus_1 += 1
            
        pauli_string_expectation_value += pow(-1, power_of_minus_1)*proba
        power_of_minus_1 = reset_power_of_minus_1(power_of_minus_1)
        
    return pauli_string_expectation_value

def get_expectation_value(pauli_string, probas_distribution):
        return calculate_expectation_value_of_pauli_string_by_measurments_probas(
                                                                                pauli_string, probas_distribution)

# Calculating the target function

In [5]:
def target_function(thetas):
    ansatz = calc_ansatz_circ(QUBITS_NUM, Entangelment, thetas, Depth)
    
    qr = QuantumRegister(QUBITS_NUM, name="qubit")
    qc = QuantumCircuit(qr)
    qc.initialize(Init_state, qc.qubits)
    
    total_expectation_value = 0
    pauli_strings, pauli_coeffs = transfrom_hamiltonian_into_pauli_string(Ham)
    for pauli_index, pauli_string in enumerate(pauli_strings):
        qc_per_pauli_string = qc.compose(ansatz)
        qc_per_pauli_string = get_z_reduction_for_pauli_string(qc_per_pauli_string, pauli_string)
        probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(qc_per_pauli_string)
        total_expectation_value += pauli_coeffs[pauli_index] * get_expectation_value(pauli_string, probas_distribution)
    print("total expectation value:")
    print(total_expectation_value)
    return total_expectation_value
        

# Implementing the VQE algorithem

In [6]:
def VQE(optimazer, initial_thetas):
    point, value, nfev = optimazer.optimize(QUBITS_NUM*2*(Depth + 1), target_function, initial_point=initial_thetas)
    base_energy = value
    print("The VQE result for the minimal energy of the Hamiltonian given is:")
    print(base_energy)

# Setting the simulator and optimazer

In [7]:
bfgs_optimizer = L_BFGS_B(maxiter=2)
cobyla_optimazer = COBYLA(maxiter=50)
simulator_backend = Aer.get_backend('aer_simulator') 

# VQE with Transverse Ising Model Hamiltonian + LINEAR Entangelment Ansatz

In [8]:
#Backend number of shots
NUM_SHOTS = 1024

QUBITS_NUM = 4
Ham = get_hamiltonian("Ising Model")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)
    

total expectation value:
2.259043869928431
total expectation value:
2.1659199846735175
total expectation value:
2.1916739166291643
total expectation value:
2.2637470558841493
total expectation value:
2.2126411637339105
total expectation value:
2.2448894826991226
total expectation value:
2.201455236572359
total expectation value:
2.216629496175176
total expectation value:
2.2068520358375485
total expectation value:
2.2211563570974495
total expectation value:
2.230732533156203
total expectation value:
2.199522259731893
total expectation value:
2.228808268561876
total expectation value:
2.2012522775820855
total expectation value:
2.2020876345391236
total expectation value:
2.224587366865098
total expectation value:
2.2547436372854226
total expectation value:
2.2010527893033967
total expectation value:
2.200896978171804
total expectation value:
2.2356261286411856
total expectation value:
2.2363127171318022
total expectation value:
2.2137341971683373
total expectation value:
2.1685544974371

total expectation value:
2.1812264867055715
total expectation value:
2.195358878639883
total expectation value:
2.2465434000895472
total expectation value:
2.2188382195765364
total expectation value:
2.165239751287848
total expectation value:
2.1863833233431103
total expectation value:
2.1873758204324756
total expectation value:
2.206994650411077
total expectation value:
2.2039634267999784
total expectation value:
2.2108612289094194
total expectation value:
2.2228248053535116
total expectation value:
2.204846171523812
total expectation value:
2.189204208839866
total expectation value:
2.2147443252840295
total expectation value:
2.1938769688882163
total expectation value:
2.194404725117143
total expectation value:
2.2315796179907124
total expectation value:
2.2422093719319443
total expectation value:
2.259353082693011
total expectation value:
2.2010136709305437
total expectation value:
2.216564613215367
total expectation value:
2.1989627097947557
total expectation value:
2.2257208031768

total expectation value:
2.2057576071290783
total expectation value:
2.2105588617325767
total expectation value:
2.2028506415653926
total expectation value:
2.170980340145617
total expectation value:
2.2070484070001344
total expectation value:
2.2575984617979796
total expectation value:
2.2181733509786494
total expectation value:
2.194517275413293
total expectation value:
2.170479482554285
total expectation value:
2.2227305229718395
total expectation value:
2.1845985183270127
total expectation value:
2.186829453861551
total expectation value:
2.1894741614519364
total expectation value:
2.2111717707104335
total expectation value:
2.169706932059012
total expectation value:
2.1692776346251774
total expectation value:
2.2467026468076776
total expectation value:
2.2031601635760447
total expectation value:
2.2383521361851364
total expectation value:
2.1722946237417875
total expectation value:
2.2160558233478036
total expectation value:
2.179679236960609
total expectation value:
2.17227159927

# VQE with Transverse Ising Model Hamiltonian + FULL Entangelment Ansatz

In [9]:
#Backend number of shots
NUM_SHOTS = 1024

QUBITS_NUM = 4
Ham = get_hamiltonian("Ising Model")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

total expectation value:
0.305120193269214
total expectation value:
0.35661109737235797
total expectation value:
0.3324719776163124
total expectation value:
0.3040770538340061
total expectation value:
0.3001175749498783
total expectation value:
0.25682931168717477
total expectation value:
0.308696608748285
total expectation value:
0.3006560931264681
total expectation value:
0.3284921250994196
total expectation value:
0.294812923838261
total expectation value:
0.3783991588763609
total expectation value:
0.3098000030750804
total expectation value:
0.31374010313990064
total expectation value:
0.31185129411675383
total expectation value:
0.24781207910034583
total expectation value:
0.26348014600665043
total expectation value:
0.28701418401844386
total expectation value:
0.35017020758493533
total expectation value:
0.3272115188073468
total expectation value:
0.293121156432803
total expectation value:
0.28981078247453196
total expectation value:
0.26162998992804587
total expectation value:
0

total expectation value:
0.2826660779071688
total expectation value:
0.33265562307286717
total expectation value:
0.3036421680404089
total expectation value:
0.33076150939037463
total expectation value:
0.3164008225331666
total expectation value:
0.33051542271176737
total expectation value:
0.2948006412895122
total expectation value:
0.22786481280509518
total expectation value:
0.33229211013121707
total expectation value:
0.3460774039031177
total expectation value:
0.3291446235684201
total expectation value:
0.29546359162756397
total expectation value:
0.3238458044327661
total expectation value:
0.27112155752203454
total expectation value:
0.2738761370526387
total expectation value:
0.30043011856336
total expectation value:
0.2992945836417984
total expectation value:
0.24419841806014475
total expectation value:
0.3716473490042372
total expectation value:
0.34736923915969464
total expectation value:
0.2837476567397894
total expectation value:
0.32769557052312737
total expectation value:

total expectation value:
0.30982175356911035
total expectation value:
0.3237313056322285
total expectation value:
0.26800196845995916
total expectation value:
0.35189902210993274
total expectation value:
0.27911746024775363
total expectation value:
0.27466623948034
total expectation value:
0.24795584768456827
total expectation value:
0.2941721656645073
total expectation value:
0.3175011610746006
total expectation value:
0.2791474888909912
total expectation value:
0.24825989976056592
total expectation value:
0.2778669146052385
total expectation value:
0.2779444545079822
total expectation value:
0.31164052152808963
total expectation value:
0.34793799345564047
total expectation value:
0.29315864852375506
total expectation value:
0.2877893462084401
total expectation value:
0.2823412768590232
total expectation value:
0.3022176157694685
total expectation value:
0.3625085235082848
total expectation value:
0.32195589521524437
total expectation value:
0.3111946305862567
total expectation value:

total expectation value:
0.33477984313048337
total expectation value:
0.2890301518053481
total expectation value:
0.34155314714596674
total expectation value:
0.28510350118030525
total expectation value:
0.32038757930757344
total expectation value:
0.323018077788512
total expectation value:
0.322877754742528
total expectation value:
0.30596677128022054
total expectation value:
0.281442005448981
total expectation value:
0.3077844975756374
total expectation value:
0.3350812172418317
total expectation value:
0.3100182714858075
total expectation value:
0.32867479234911406
total expectation value:
0.3264920020589818
total expectation value:
0.31456355902638483
total expectation value:
0.349257467125367
total expectation value:
0.3175796056205329
total expectation value:
0.27888202930401806
total expectation value:
0.3125918981502034
total expectation value:
0.2743556249624962
total expectation value:
0.2842255936182011
total expectation value:
0.268601753272975
total expectation value:
0.27

total expectation value:
0.2551955773033063
total expectation value:
0.27167202202814067
total expectation value:
0.30291852068749237
total expectation value:
0.3418277468487796
total expectation value:
0.3145038186366738
total expectation value:
0.23114223529237019
total expectation value:
0.34303545358440674
total expectation value:
0.22883044213776577
total expectation value:
0.2593278814841955
total expectation value:
0.2983927297521436
total expectation value:
0.28209683531596885
total expectation value:
0.30710590996431486
total expectation value:
0.3037634594240227
total expectation value:
0.291950046624338
total expectation value:
0.2958928076693029
total expectation value:
0.330742271797545
total expectation value:
0.3551800765558659
total expectation value:
0.24568763914624084
total expectation value:
0.3078181025790595
total expectation value:
0.33395072102977597
total expectation value:
0.3415965854623948
total expectation value:
0.3361035575353445
total expectation value:


# VQE with H2 Molecule Hamiltonian + LINEAR Entangelment Ansatz

In [10]:
#Backend number of shots
NUM_SHOTS = 1024

QUBITS_NUM = 2
Ham = get_hamiltonian("Molecular")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

total expectation value:
-0.31002254306482074
total expectation value:
-0.2925154148197847
total expectation value:
-0.30256524759025216
total expectation value:
-0.3064378633442672
total expectation value:
-0.2985645201109158
total expectation value:
-0.27795285286659427
total expectation value:
-0.30634190939966327
total expectation value:
-0.3153514743738747
total expectation value:
-0.31187886458810843
total expectation value:
-0.31248696863539843
total expectation value:
-0.3123269217283091
total expectation value:
-0.30989450553914927
total expectation value:
-0.3101505805904922
total expectation value:
-0.29738032139775383
total expectation value:
-0.31306306330127054
total expectation value:
-0.30403745653652536
total expectation value:
-0.2990445866325343
total expectation value:
-0.641489229200682
total expectation value:
-0.6311513861971033
total expectation value:
-0.5868396286993579
total expectation value:
-0.6211656463891212
total expectation value:
-0.6173729427891763
t

total expectation value:
-0.3026772433253898
total expectation value:
-0.2905150881799413
total expectation value:
-0.3051897200679191
total expectation value:
-0.2810254566841115
total expectation value:
-0.3212244909673827
total expectation value:
-0.27756881448922943
total expectation value:
-0.2881146813721995
total expectation value:
-0.28829069587017286
total expectation value:
-0.28064134410709696
total expectation value:
-0.3269054512723836
total expectation value:
-0.27612854072472437
total expectation value:
-0.3166796782035923
total expectation value:
-0.2767686541534322
total expectation value:
-0.28757059608774527
total expectation value:
-0.30410147529936105
total expectation value:
-0.32725762866762986
total expectation value:
-0.29323558880186196
total expectation value:
-0.30879029317970713
total expectation value:
-0.2834098217013196
total expectation value:
-0.30818218913241724
total expectation value:
-0.31949620696976655
total expectation value:
-0.2870424783941751

total expectation value:
-0.3097184910411758
total expectation value:
-0.3070620091820909
total expectation value:
-0.29910860539537
total expectation value:
-0.32312886366262206
total expectation value:
-0.31723980527858026
total expectation value:
-0.314327322567802
total expectation value:
-0.3042774897973345
total expectation value:
-0.31079069401920006
total expectation value:
-0.3146633097732152
The VQE result for the minimal energy of the Hamiltonian given is:
-0.311126755424263


# VQE with H2 Molecule Hamiltonian + FULL Entangelment Ansatz

In [11]:
#Backend number of shots
NUM_SHOTS = 1024

QUBITS_NUM = 2
Ham = get_hamiltonian("Molecular")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

total expectation value:
-0.32985001756422927
total expectation value:
-0.3297539894199757
total expectation value:
-0.3161836440910897
total expectation value:
-0.3065179238976367
total expectation value:
-0.28312181146820836
total expectation value:
-0.2992046335396237
total expectation value:
-0.29544393932109664
total expectation value:
-0.3196561796772063
total expectation value:
-0.31264701554248775
total expectation value:
-0.3022451537760734
total expectation value:
-0.3244090905200377
total expectation value:
-0.33244248066047843
total expectation value:
-0.30338137551693334
total expectation value:
-0.3032373704003778
total expectation value:
-0.29893259089739654
total expectation value:
-0.2877786199671366
total expectation value:
-0.30837424542092445
total expectation value:
-0.2275602774948984
total expectation value:
-0.2152220336518268
total expectation value:
-0.2300406706563602
total expectation value:
-0.2056844251836949
total expectation value:
-0.2381541213501704
to

total expectation value:
-0.2814415786425438
total expectation value:
-0.31152676139251184
total expectation value:
-0.2968041525322321
total expectation value:
-0.3164236031522493
total expectation value:
-0.32276079287614134
total expectation value:
-0.31988031954678103
total expectation value:
-0.30477359810948673
total expectation value:
-0.3057337311527237
total expectation value:
-0.30008478022914065
total expectation value:
-0.2976523640399809
total expectation value:
-0.27625657825039596
total expectation value:
-0.29910860539537
total expectation value:
-0.3178479093258702
total expectation value:
-0.31496736179686025
total expectation value:
-0.288818813563743
total expectation value:
-0.31285503942187914
total expectation value:
-0.29994070091293545
total expectation value:
-0.28969896025326003
total expectation value:
-0.2893308894667793
total expectation value:
-0.29758834527714517
total expectation value:
-0.30733405182431806
total expectation value:
-0.30194110175242844


# VQE with Transverse Ising Model Hamiltonian + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [12]:
#Backend number of shots
NUM_SHOTS = 1024

QUBITS_NUM = 4
Ham = get_hamiltonian("Ising Model")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)
    

total expectation value:
1.4139768434707383
total expectation value:
1.8189514784170544
total expectation value:
1.6299590649777929
total expectation value:
1.5920722511296792
total expectation value:
1.749444182789571
total expectation value:
1.5221527595481514
total expectation value:
1.6201548354068942
total expectation value:
1.6469494104310256
total expectation value:
1.5571421380837596
total expectation value:
1.6695598504656184
total expectation value:
1.632561662336268
total expectation value:
1.3345505984380568
total expectation value:
1.5120786488356013
total expectation value:
1.5201724382006392
total expectation value:
1.7409331005598887
total expectation value:
1.8220596231488668
total expectation value:
1.6809486442037758
total expectation value:
1.7807903452211375
total expectation value:
1.680562771511189
total expectation value:
1.3104347825692555
total expectation value:
1.4334413710585208
total expectation value:
1.5204731486904461
total expectation value:
1.26096964

# VQE with Transverse Ising Model Hamiltonian + FULL Entangelment Ansatz Using COBYLA Optimazer

In [13]:
#Backend number of shots
NUM_SHOTS = 1024

QUBITS_NUM = 4
Ham = get_hamiltonian("Ising Model")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

total expectation value:
2.4866753694021835
total expectation value:
2.715843160738209
total expectation value:
2.3718288296971437
total expectation value:
2.276417707912527
total expectation value:
2.4054126023138753
total expectation value:
2.318636898121994
total expectation value:
2.3129070657682567
total expectation value:
2.340414707924717
total expectation value:
2.2968894158494373
total expectation value:
2.0607662517236474
total expectation value:
2.395604600719075
total expectation value:
2.802031185343013
total expectation value:
1.961233558700412
total expectation value:
1.9721747986608087
total expectation value:
1.8908761200370066
total expectation value:
1.993377019423508
total expectation value:
1.9432547332607606
total expectation value:
2.225882853023819
total expectation value:
1.9527448817189026
total expectation value:
2.2663643268281666
total expectation value:
1.9345677042382332
total expectation value:
1.9853674924794271
total expectation value:
1.98676599947322

# VQE with H2 Molecule Hamiltonian + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [14]:
#Backend number of shots
NUM_SHOTS = 1024

QUBITS_NUM = 2
Ham = get_hamiltonian("Molecular")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

total expectation value:
-0.32759361587304303
total expectation value:
-0.050856887590473715
total expectation value:
-0.4080555548031221
total expectation value:
-0.3720011789160931
total expectation value:
-0.6222853069419001
total expectation value:
-0.540671549678325
total expectation value:
-0.5835105788322498
total expectation value:
-0.5492963342786754
total expectation value:
-0.5264763907147688
total expectation value:
-0.46200109731255207
total expectation value:
-0.7868588478808927
total expectation value:
-0.7435869871811552
total expectation value:
-0.7773211652131111
total expectation value:
-0.7587741401700671
total expectation value:
-0.6597160862298976
total expectation value:
-0.7730644368820815
total expectation value:
-0.7790334074201937
total expectation value:
-0.7511248626066408
total expectation value:
-0.8107513654210716
total expectation value:
-0.8043982081062954
total expectation value:
-0.7605185401578666
total expectation value:
-0.7903475736568422
total e

# VQE with H2 Molecule Hamiltonian + FULL Entangelment Ansatz Using COBYLA Optimazer

In [15]:
#Backend number of shots
NUM_SHOTS = 1024

QUBITS_NUM = 2
Ham = get_hamiltonian("Molecular")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
Stop_optimization_threshold = 100
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

total expectation value:
-0.3014930446122278
total expectation value:
-0.04074311025682026
total expectation value:
-0.3817629272537997
total expectation value:
-0.4033827045136601
total expectation value:
-0.6273261538181931
total expectation value:
-0.5513134447055485
total expectation value:
-0.5326056307589191
total expectation value:
-0.5678915588929703
total expectation value:
-0.5163146364088131
total expectation value:
-0.47329907335936766
total expectation value:
-0.7756888351600988
total expectation value:
-0.7211510077949639
total expectation value:
-0.7973406960010267
total expectation value:
-0.8754992208630628
total expectation value:
-0.8269948279967732
total expectation value:
-0.8730988140553209
total expectation value:
-0.8656095091993342
total expectation value:
-0.6705348861508903
total expectation value:
-0.949624489466797
total expectation value:
-0.9556417337758096
total expectation value:
-0.9184671037388047
total expectation value:
-0.9049767447636385
total exp